In [1]:
import torch.nn as nn
import torch.nn.functional as F
import net_size_utils as nsu


class Net(nn.Module):
    def __init__(self, kernel_size=5, out_features=10):
        super().__init__()
        self.kernel_size = kernel_size
        self.conv1 = nn.Conv2d(3, 6, kernel_size)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size)
        conv_height, conv_width = self.conv_out_size(224, 224)
        fc1_in = 16 * conv_width * conv_height
        fc1_out = 1000 #int((fc1_in*out_features)**(1/2))
        self.fc1 = nn.Linear(fc1_in, fc1_out)
        self.fc2 = nn.Linear(fc1_out, out_features)
        print(f"input layer: {fc1_in}, hidden: {fc1_out}, output: {out_features}")

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
    def conv_out_size(self, height, width):
        dim = (height, width)
        dim = nsu.dim_conv2d(dim, self.kernel_size)
        dim = nsu.dim_maxpool2d(dim, 2, 2)
        dim = nsu.dim_conv2d(dim, self.kernel_size)
        dim = nsu.dim_maxpool2d(dim, 2, 2)
        return dim


In [2]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False


In [3]:
from birds_dataset import Birds270Dataset

import net_training
import torch
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision import datasets, models

dataset_dir = "../data/birds270"
train_transform= transforms.Compose([
    transforms.Normalize((127.5, 127.5, 127.5), (127.5, 127.5, 127.5))
])
test_transform = transforms.Normalize((127.5, 127.5, 127.5), (127.5, 127.5, 127.5)) # normalizes colors to range [-1,1]
train_dataset = Birds270Dataset(dataset_dir, set_type="train", selected_birds=None, transform=train_transform)
test_dataset = Birds270Dataset(dataset_dir, set_type=["test","valid"], selected_birds=None, transform=test_transform)

label_set = test_dataset.get_label_set()

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)


net = Net(out_features=270)
optimizer = optim.Adam(net.parameters(), lr=0.001)


net_training.train_and_evaluate(net, train_dataloader, test_dataloader,
                                label_set, epochs=200, optimizer=optimizer, print_results=True)


input layer: 44944, hidden: 1000, output: 270
Epoch 0:
	train loss: 4.036752193985132
	validation loss: 2.5737901101288974, validation accuracy: 40.88888888888889%
	Elapsed time: 0:10:34.374182
Epoch 1:
	train loss: 1.920458342577291
	validation loss: 2.0491013113657632, validation accuracy: 51.11111111111111%
	Elapsed time: 0:11:53.260927
Epoch 2:
	train loss: 0.3508847385166751
	validation loss: 2.7040046370470967, validation accuracy: 48.55555555555556%
	Elapsed time: 0:11:58.098262
Epoch 3:
	train loss: 0.09164903618441171
	validation loss: 3.361210661464267, validation accuracy: 48.25925925925926%
	Elapsed time: 0:11:46.671233
Epoch 4:
	train loss: 0.10313947315270906
	validation loss: 4.071830134744998, validation accuracy: 44.592592592592595%
	Elapsed time: 0:11:25.530946
Epoch 5:
	train loss: 0.093233367960095
	validation loss: 4.319851560592651, validation accuracy: 44.592592592592595%
	Elapsed time: 0:12:56.403122
Epoch 6:
	train loss: 0.07751106072202815
	validation loss: 4.

KeyboardInterrupt: 